In [ ]:
import random
import os
import cv2

import shutil
import tqdm
import glob
from ultralytics import YOLO

import os
import psycopg2
from psycopg2.extras import execute_batch


In [ ]:


# List all files in the directory
image_files = [f for f in os.listdir(images_directory) if f.endswith('.jpg')]

# Iterate over each file in the directory
for image_file in image_files:
    image_path = os.path.join(images_directory, image_file)  # Construct full image path
    # Perform prediction
    predictions = detection_model.predict(source=image_path, conf=0.5, save=True, line_width=2, show_labels=True, save_crop=True)

base_path = 'runs/detect/predict2/crops/'
clothing_types = ['bag', 'dress', 'hat', 'jacket', 'pant', 'shirt', 'shoe', 'short', 'skirt']

for clothing_type in clothing_types:
    folder_path = os.path.join(base_path, clothing_type)
    if os.path.exists(folder_path):
        items_to_insert = []  # Accumulate items to insert in batches
        for image_name in os.listdir(folder_path):
            if image_name.endswith('.jpg'):  # Check if it's an image
                image_path = os.path.join(folder_path, image_name)
                items_to_insert.append((clothing_type, image_path))
                
        # Insert the accumulated items into the database
        if items_to_insert:
            insert_items(items_to_insert)
            print(f"Added {len(items_to_insert)} items of type '{clothing_type}' to the database.")

# Close the database connection
cur.close()
conn.close()


In [ ]:
conn = psycopg2.connect(**db_params)
cur = conn.cursor()

# Execute the SELECT statement
cur.execute("""
SELECT * FROM clothing_items;
""")

# Fetch all rows from the table
rows = cur.fetchall()

# Print the rows
for row in rows:
    print(row)

# Close the database connection
cur.close()
conn.close()


In [16]:
import psycopg2
from psycopg2.extras import execute_batch
import os
import shutil
from ultralytics import YOLO

# Set database connection environment variables
os.environ['DATABASE_NAME'] = 'mywardrobe'
os.environ['DATABASE_USER'] = 'postgres'
os.environ['DATABASE_PASSWORD'] = 'mysecretpassword'
os.environ['DATABASE_HOST'] = 'my-postgres'  # Change to 'localhost' if directly connected to PostgreSQL without Docker

# Database connection parameters
db_params = {
    "dbname": os.getenv("DATABASE_NAME"),
    "user": os.getenv("DATABASE_USER"),
    "password": os.getenv("DATABASE_PASSWORD"),
    "host": os.getenv("DATABASE_HOST")
}

# Connect to your PostgreSQL database
conn = psycopg2.connect(**db_params)
cur = conn.cursor()

# Create tables for each type of clothing
clothing_types = ['bag', 'dress', 'hat', 'jacket', 'pant', 'shirt', 'shoe', 'short', 'skirt']
for clothing_type in clothing_types:
    cur.execute(f"""
    CREATE TABLE IF NOT EXISTS {clothing_type} (
        id SERIAL PRIMARY KEY,
        image_url VARCHAR(255)
    );
    """)
conn.commit()

# Function to insert items into the database, modified to handle different tables
def insert_items(clothing_type, items):
    query = f"INSERT INTO {clothing_type} (image_url) VALUES (%s);"
    execute_batch(cur, query, [(item,) for item in items])
    conn.commit()

# Load pre-trained model
detection_model = YOLO("best.pt")

# Directory containing the images
images_directory = 'ClothingRecommendation/images/'
crop_directory = 'runs/detect/predict'

# Ensure the target crops folder exists
target_crops_folder = 'crops'
if not os.path.exists(target_crops_folder):
    os.makedirs(target_crops_folder)

# Process images and move crops to a common folder, independent of runs
image_files = [f for f in os.listdir(images_directory) if f.endswith('.jpg')]
for image_file in image_files:
    image_path = os.path.join(images_directory, image_file)
    predictions = detection_model.predict(source=image_path, conf=0.5, save=True, line_width=2, show_labels=True, save_crop=True)

    # New logic to handle common crops folder
    for clothing_type in clothing_types:
        source_crops_folder = os.path.join(target_crops_folder, clothing_type)
        source_crops_folder_item = os.path.join(crop_directory, source_crops_folder)
        if os.path.exists(source_crops_folder_item):
            items_to_insert = []
            for crop_img in os.listdir(source_crops_folder_item):
                source_path = os.path.join(source_crops_folder_item, crop_img)
                destination_path = os.path.join(target_crops_folder, clothing_type, crop_img)
                if not os.path.exists(os.path.dirname(destination_path)):
                    os.makedirs(os.path.dirname(destination_path))
                shutil.move(source_path, destination_path)
                items_to_insert.append(destination_path)

            # Enter into type of clothe the given clothe
            if items_to_insert:
                insert_items(clothing_type, items_to_insert)

    # Remove predict directory after entering all the predict crop in my database            
    if os.path.exists(crop_directory):
        shutil.rmtree(crop_directory)

# Close the database connection
cur.close()
conn.close()



image 1/1 /usr/src/app/ClothingRecommendation/images/163.jpg: 640x448 1 jacket, 1 pants, 1 bag, 1 shoe, 231.0ms
Speed: 11.8ms preprocess, 231.0ms inference, 8.7ms postprocess per image at shape (1, 3, 640, 448)
Results saved to runs/detect/predict
crops/bag
Added 1 items of type 'bag' to the database.
crops/dress
crops/hat
crops/jacket
Added 1 items of type 'jacket' to the database.
crops/pant
crops/shirt
crops/shoe
Added 1 items of type 'shoe' to the database.
crops/short
crops/skirt

image 1/1 /usr/src/app/ClothingRecommendation/images/60.jpg: 640x448 1 hat, 1 shirt, 1 shorts, 1 shoe, 196.3ms
Speed: 4.9ms preprocess, 196.3ms inference, 9.1ms postprocess per image at shape (1, 3, 640, 448)
Results saved to runs/detect/predict
crops/bag
crops/dress
crops/hat
Added 1 items of type 'hat' to the database.
crops/jacket
crops/pant
crops/shirt
Added 1 items of type 'shirt' to the database.
crops/shoe
Added 1 items of type 'shoe' to the database.
crops/short
crops/skirt

image 1/1 /usr/src/a

In [15]:
# Connect to your PostgreSQL database
conn = psycopg2.connect(**db_params)
cur = conn.cursor()

# Query to fetch all table names in the 'public' schema of your database
cur.execute("""
SELECT table_name
FROM information_schema.tables
WHERE table_schema = 'public'
ORDER BY table_name;
""")

# Fetch all rows from the query
tables = cur.fetchall()

# Print the names of the tables and their data
for table in tables:
    table_name = table[0]  # Correctly extract the table name from the tuple
    try:
        cur.execute(f"SELECT * FROM {table_name};")  # Use the extracted table name in the query
        rows = cur.fetchall()
        print(f"Data from table: {table_name}")
        for row in rows:
            print(row)
    except Exception as e:
        print(f"An error occurred: {e}")

# Close the database connection
cur.close()
conn.close()


Data from table: bag
(1, 'crops/bag/163.jpg')
(2, 'crops/bag/163.jpg')
(3, 'crops/bag/1.jpg')
Data from table: dress
(1, 'crops/dress/8.jpg')
(2, 'crops/dress/34.jpg')
(3, 'crops/dress/82.jpg')
(4, 'crops/dress/1.jpg')
Data from table: hat
(1, 'crops/hat/60.jpg')
Data from table: jacket
(1, 'crops/jacket/163.jpg')
(2, 'crops/jacket/163.jpg')
(3, 'crops/jacket/8.jpg')
Data from table: pant
Data from table: shirt
(1, 'crops/shirt/60.jpg')
(2, 'crops/shirt/42.jpg')
(3, 'crops/shirt/51.jpg')
(4, 'crops/shirt/1.jpg')
Data from table: shoe
(1, 'crops/shoe/163.jpg')
(2, 'crops/shoe/163.jpg')
(3, 'crops/shoe/60.jpg')
(4, 'crops/shoe/8.jpg')
(5, 'crops/shoe/107.jpg')
(6, 'crops/shoe/34.jpg')
(7, 'crops/shoe/82.jpg')
(8, 'crops/shoe/1.jpg')
Data from table: short
Data from table: skirt
(1, 'crops/skirt/71.jpg')


In [6]:
conn = psycopg2.connect(**db_params)
cur = conn.cursor()

# Drop the clothing_items table
cur.execute("DROP TABLE IF EXISTS clothing_items;")
conn.commit()

cur.close()
conn.close()

In [17]:
if os.path.exists(crop_directory):
    shutil.rmtree(crop_directory)
    print(f"Removed directory: {crop_directory}")

Removed directory: runs/detect/predict
